## Speckgürtel-Projekt

### Import

In [1]:
import pandas as pd
import numpy as np
import warnings
import json
import os
import glob
from sklearn.preprocessing import StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

### Load

In [2]:
# Load the first dataset
df = pd.read_csv('data/Gemergte Daten vorläufig.csv')

df.columns = [
    'id', 'name', 'metropole', 'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner'
]

# Convert columns from km to m
columns_to_convert_1 = ['autobahn', 'zug', 'supermarkt']
df[columns_to_convert_1] = df[columns_to_convert_1] * 1000

columns_to_convert_2 = ['preis_entwicklung', 'angebot_entwicklung']
df[columns_to_convert_2] = df[columns_to_convert_2] * 100

# Load the second dataset
df_gesundheit = pd.read_excel('data/HausärzteNachKreisen-Deutschlandatlas.xlsx')

# Rename columns in the second dataset for clarity
df_gesundheit.columns = ['id', 'kreisname', 'gesundheit']

# Merge the datasets on the 'id' column
df = pd.merge(df, df_gesundheit[['id', 'gesundheit']], on='id', how='left')

# Display the first few rows of the merged DataFrame to verify
df.head(5)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,schule,einwohner,gesundheit
0,9173,Bad Tölz-Wolfratshausen,München,5689,391.0,6.715438,-5.555556,12550.0,4370.0,2150.0,16.863591,75.390603,7.560664,65.1
1,12060,Barnim,Berlin,3317,535.0,-2.869693,-25.694444,6810.0,2960.0,2580.0,41.134607,62.160062,11.307039,53.4
2,11000,Berlin,Berlin,5527,13095.0,0.857664,-25.996044,4470.0,1840.0,590.0,2.972945,58.087855,8.507855,63.1
3,5314,Bonn,Köln,3860,948.0,4.127327,-21.327801,2360.0,2400.0,720.0,13.774530,91.426461,3.811611,62.4
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.984413,-38.608696,13390.0,3870.0,3450.0,1.501034,60.392608,-5.838318,64.0


In [3]:
df_schule_neu=pd.read_excel('data/SchulenJe100_000.xlsx')
df_schule_neu_red=df_schule_neu[["ID", "Schulen pro 100.000 Einwohner"]]
df_schule_neu_red=df_schule_neu_red.rename(columns={"ID": "id", "Schulen pro 100.000 Einwohner": "schule_neu"})
df_schule_neu_red


,id,schule_neu
0,1053,38.079244
1,1056,44.441975
2,1060,39.000609
3,1062,34.237333
4,2000,34.709134
...,...,...
71,14729,37.083338
72,14730,43.067185
73,15002,31.795583
74,15084,42.331268


In [4]:
df_merged=pd.merge(df, df_schule_neu_red, on='id', how='left')
df_merged.drop(["schule"], axis=1, inplace=True)
df_merged.rename(columns={"schule_neu": "schule"}, inplace=True)
df_merged

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule
0,9173,Bad Tölz-Wolfratshausen,München,5689,391.0,6.715438,-5.555556,12550.0,4370.0,2150.0,16.863591,7.560664,65.1,40.712234
1,12060,Barnim,Berlin,3317,535.0,-2.869693,-25.694444,6810.0,2960.0,2580.0,41.134607,11.307039,53.4,52.836053
2,11000,Berlin,Berlin,5527,13095.0,0.857664,-25.996044,4470.0,1840.0,590.0,2.972945,8.507855,63.1,34.556589
3,5314,Bonn,Köln,3860,948.0,4.127327,-21.327801,2360.0,2400.0,720.0,13.774530,3.811611,62.4,29.482800
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.984413,-38.608696,13390.0,3870.0,3450.0,1.501034,-5.838318,64.0,42.331268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,12072,Teltow-Fläming,Berlin,3119,482.0,-1.918239,-29.428990,15150.0,3350.0,2890.0,30.731746,11.105545,54.8,50.985534
81,8416,Tübingen,Stuttgart,3992,605.0,4.148187,-18.900804,17280.0,2460.0,1910.0,7.003337,8.377619,59.8,43.469182
82,6440,Wetteraukreis,Frankfurt,3178,864.0,3.181818,-31.374106,7140.0,2980.0,1990.0,23.899081,5.703545,64.0,42.064421
83,6414,Wiesbaden,Frankfurt,4476,1083.0,8.667152,-31.063017,3030.0,2510.0,940.0,13.907001,5.356959,66.1,37.475221


In [5]:
Landkreisnamen=pd.read_excel('data/Landkreisnamen.xlsx')
Landkreisnamen=Landkreisnamen.rename(columns={"Kreiskennziffer": "id"})
Landkreisnamen.id=pd.to_numeric(Landkreisnamen.id)
Landkreisnamen=Landkreisnamen[["id", "Stadt/Landkreis"]]
Landkreisnamen

,id,Stadt/Landkreis
0,9173,Bad Tölz-Wolfratshausen
1,12060,Barnim
2,11000,Berlin
3,5314,Bonn
4,15084,Burgenlandkreis
...,...,...
80,12072,Teltow-Fläming
81,8416,Tübingen
82,6440,Wetteraukreis
83,6414,Wiesbaden


In [6]:
df_merged[df_merged["name"].str.contains("Leipzig")]

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule
38,14713,Leipzig,Leipzig,3114,2546.0,4.182001,-31.43011,7180.0,1440.0,640.0,4.193158,16.737450,61.4,27.908672
39,14729,Leipzig,Leipzig,2026,801.0,-3.477847,-29.92126,7840.0,3060.0,3160.0,2.032328,0.578302,71.5,37.083338


In [7]:
df_merged_lk=pd.merge(df_merged, Landkreisnamen, on='id', how='inner')
df_merged_lk=df_merged_lk.drop_duplicates()
df_merged_lk["name"]=df_merged_lk["Stadt/Landkreis"]
df_merged_lk

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule,Stadt/Landkreis
0,9173,Bad Tölz-Wolfratshausen,München,5689,391.0,6.715438,-5.555556,12550.0,4370.0,2150.0,16.863591,7.560664,65.1,40.712234,Bad Tölz-Wolfratshausen
1,12060,Barnim,Berlin,3317,535.0,-2.869693,-25.694444,6810.0,2960.0,2580.0,41.134607,11.307039,53.4,52.836053,Barnim
2,11000,Berlin,Berlin,5527,13095.0,0.857664,-25.996044,4470.0,1840.0,590.0,2.972945,8.507855,63.1,34.556589,Berlin
3,5314,Bonn,Köln,3860,948.0,4.127327,-21.327801,2360.0,2400.0,720.0,13.774530,3.811611,62.4,29.482800,Bonn
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.984413,-38.608696,13390.0,3870.0,3450.0,1.501034,-5.838318,64.0,42.331268,Burgenlandkreis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,12072,Teltow-Fläming,Berlin,3119,482.0,-1.918239,-29.428990,15150.0,3350.0,2890.0,30.731746,11.105545,54.8,50.985534,Teltow-Fläming
99,8416,Tübingen,Stuttgart,3992,605.0,4.148187,-18.900804,17280.0,2460.0,1910.0,7.003337,8.377619,59.8,43.469182,Tübingen
100,6440,Wetteraukreis,Frankfurt,3178,864.0,3.181818,-31.374106,7140.0,2980.0,1990.0,23.899081,5.703545,64.0,42.064421,Wetteraukreis
101,6414,Wiesbaden,Frankfurt,4476,1083.0,8.667152,-31.063017,3030.0,2510.0,940.0,13.907001,5.356959,66.1,37.475221,Wiesbaden


In [8]:
df_merged_lk[df_merged_lk["name"].str.contains("Leipzig")]

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule,Stadt/Landkreis
44,14713,Leipzig,Leipzig,3114,2546.0,4.182001,-31.43011,7180.0,1440.0,640.0,4.193158,16.737450,61.4,27.908672,Leipzig
45,14729,Landkreis Leipzig,Leipzig,2026,801.0,-3.477847,-29.92126,7840.0,3060.0,3160.0,2.032328,0.578302,71.5,37.083338,Landkreis Leipzig


In [9]:
df_merged_lk.drop(["Stadt/Landkreis"], axis=1, inplace=True)
df=df_merged_lk

In [10]:
df

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule
0,9173,Bad Tölz-Wolfratshausen,München,5689,391.0,6.715438,-5.555556,12550.0,4370.0,2150.0,16.863591,7.560664,65.1,40.712234
1,12060,Barnim,Berlin,3317,535.0,-2.869693,-25.694444,6810.0,2960.0,2580.0,41.134607,11.307039,53.4,52.836053
2,11000,Berlin,Berlin,5527,13095.0,0.857664,-25.996044,4470.0,1840.0,590.0,2.972945,8.507855,63.1,34.556589
3,5314,Bonn,Köln,3860,948.0,4.127327,-21.327801,2360.0,2400.0,720.0,13.774530,3.811611,62.4,29.482800
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.984413,-38.608696,13390.0,3870.0,3450.0,1.501034,-5.838318,64.0,42.331268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,12072,Teltow-Fläming,Berlin,3119,482.0,-1.918239,-29.428990,15150.0,3350.0,2890.0,30.731746,11.105545,54.8,50.985534
99,8416,Tübingen,Stuttgart,3992,605.0,4.148187,-18.900804,17280.0,2460.0,1910.0,7.003337,8.377619,59.8,43.469182
100,6440,Wetteraukreis,Frankfurt,3178,864.0,3.181818,-31.374106,7140.0,2980.0,1990.0,23.899081,5.703545,64.0,42.064421
101,6414,Wiesbaden,Frankfurt,4476,1083.0,8.667152,-31.063017,3030.0,2510.0,940.0,13.907001,5.356959,66.1,37.475221


In [11]:
df.to_csv("Daten_Stand_1908.csv", index=False)

### Standardize

Wir standisieren, um den Einfluss aller Features auf den Score anzugleichen. Die Ergebnisse speichern wir in ein Dictionary of Dataframes, indem für jede Stadt ein Dataframe hinterlegt ist.

In [12]:

# Spalten, die standardisiert werden sollen
columns_to_standardize = [
    'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner', 'gesundheit'
]





### Save

Die Datei soll an das Frontend geliefert werden. Ebenfalls enthalten sind Spalten-Mediane und -Varianzen zur späteren Standardisierung des Userinputs.  

In [13]:
# Verzeichnis mit den JSON-Dateien
input_directory = 'data/city/'
output_directory = 'export/'

# Lade alle JSON-Dateien, die dem Muster data_[].json entsprechen
json_files = glob.glob(os.path.join(input_directory, 'data_*.json'))

# Iteriere über alle JSON-Dateien
for json_file in json_files:
    # Lade die Daten aus der aktuellen JSON-Datei
    with open(json_file, 'r') as file:
        city_data = json.load(file)

    # Extrahiere die Metropole (city) aus den Daten
    target_city = city_data['city']
    print(f"Verarbeite Metropole: {target_city}")

    # Filtere die Kreise, die zur Metropole gehören
    filtered_df = df[df["metropole"]==target_city] # Originalwerte
    # DataFrame kopieren
    filtered_df_scaled = filtered_df.copy()

    # Standardisierung durchführen
    scaler = StandardScaler()
    filtered_df_scaled[columns_to_standardize] = scaler.fit_transform(filtered_df[columns_to_standardize])

    # Ergebnis anzeigen
    filtered_df.head(5)
    kreise = []
    for (_, row_original), (_, row_scaled) in zip(filtered_df.iterrows(), filtered_df_scaled.iterrows()):
        kreis_data = {
            "ags": int(row_original['id']),  # Verwende die ID als AGS
            "label": row_original['name'],  # Kreisname
            "data": []
        }
        for feature in ['preis', 'angebot', 'preis_entwicklung', 'angebot_entwicklung', 
                        'autobahn', 'zug', 'supermarkt', 'pendler', 'schule', 'einwohner', 'gesundheit']:


        # Berechne chartValue nur für bestimmte Features
            if feature in ['schule', 'angebot', 'pendler', 'einwohner']:
                column_min = df[feature].min()
                column_max = df[feature].max()
                chart_value = (row_original[feature] - column_min) / (column_max - column_min)
            else:
                chart_value = None  # Kein chartValue für andere Features

            kreis_data["data"].append({
                "id": feature,
                "displayValue": row_original[feature],  # Originalwert aus df
                "score": row_scaled[feature],  # Standardisierter Wert aus df_scaled
                **({"chartValue": chart_value} if chart_value is not None else {})  # Füge chartValue nur hinzu, wenn berechnet
            })
        kreise.append(kreis_data)

    # Erstelle die Statistikdaten als Liste von Objekten
        statistics = [
            {
                "id": feature,
                "means": scaler.mean_[columns_to_standardize.index(feature)],
            "sd": np.sqrt(scaler.var_[columns_to_standardize.index(feature)])
            }
            for feature in ['preis', 'angebot', 'preis_entwicklung', 'angebot_entwicklung', 
                            'autobahn', 'zug', 'supermarkt', 'pendler', 'schule', 'einwohner', 'gesundheit']
        ]
    barchart= [
    {
      "id": "preis",
      "unit": "euro",
      "min": 0,
      "max": 3114,
      "labels": ["sliderInput"],
      "barStartingPoint": "min"
    },
    {
      "id": "angebot",
      "unit": "count",
      "min": 0,
      "max": 2546,
      "median": 477,
      "labels": ["median"],
      "barStartingPoint": "min"
    },
    {
      "id": "preis_entwicklung",
      "unit": "growth",
      "min": -3.47784659361601,
      "max": 4.18200066912011,
      "labels": ["0", "max"],
      "barStartingPoint": 0
    },
    {
      "id": "angebot_entwicklung",
      "unit": "growth",
      "min": -38.6086956521739,
      "max": -27.373417721519,
      "labels": ["min"],
      "barStartingPoint": "max"
    },
    {
      "id": "autobahn",
      "unit": "distance",
      "min": 0,
      "minVal": 6290,
      "max": 19630,
      "labels": ["min"],
      "barStartingPoint": 0
    },
    {
      "id": "zug",
      "unit": "distance",
      "min": 0,
      "minVal": 1440,
      "max": 4800,
      "labels": ["min"],
      "barStartingPoint": 0
    },
    {
      "id": "supermarkt",
      "unit": "distance",
      "min": 0,
      "minVal": 640,
      "max": 3880,
      "labels": ["min"],
      "barStartingPoint": 0
    },
    {
      "id": "pendler",
      "unit": "growth",
      "min": 0,
      "max": 4.19315773539207,
      "median": 2.122814294,
      "labels": ["median"],
      "barStartingPoint": "min"
    },
    {
      "id": "schule",
      "unit": "count",
      "min": 0,
      "max": 70.2686720756905,
      "median": 62.358175625,
      "labels": ["median"],
      "barStartingPoint": "min"
    },
    {
      "id": "einwohner",
      "unit": "growth",
      "min": -16.7374500324477,
      "max": 16.7374500324477,
      "labels": ["-", "0", "+"],
      "barStartingPoint": 0
    },
    {
      "id": "gesundheit",
      "unit": "count",
      "min": 0,
      "max": 71.5,
      "labels": ["max"],
      "barStartingPoint": "min"
    }
  ]

    slider= [{ "id": "preis", "min": 1200, "max": 3200 }]

    # Erstelle die finale JSON-Struktur, inklusive zusätzlicher Werte aus der aktuellen JSON-Datei
    output_data = {
        "city": city_data['city'],
        "cityAGS": city_data.get("cityAGS"),  # Übernehme cityAGS
        "map": city_data.get("map"),  # Übernehme map-Daten
        "statistics": statistics, 
         "barchart": barchart, 
          "slider":slider, # Statistikdaten als Liste von Objekten
        "kreise": kreise
    }

    # Speichere die JSON-Datei
    output_file = os.path.join(output_directory, f'data_{target_city.lower()}.json')
    os.makedirs(output_directory, exist_ok=True) 
    with open(output_file, 'w',  encoding='utf-8') as file:
        json.dump(output_data, file, indent=4, )

    print(f"JSON-Datei für {target_city} wurde erfolgreich unter {output_file} gespeichert.")

Verarbeite Metropole: Leipzig
JSON-Datei für Leipzig wurde erfolgreich unter export/data_leipzig.json gespeichert.
Verarbeite Metropole: Stuttgart
JSON-Datei für Stuttgart wurde erfolgreich unter export/data_stuttgart.json gespeichert.
Verarbeite Metropole: Berlin
JSON-Datei für Berlin wurde erfolgreich unter export/data_berlin.json gespeichert.
Verarbeite Metropole: Düsseldorf
JSON-Datei für Düsseldorf wurde erfolgreich unter export/data_düsseldorf.json gespeichert.
Verarbeite Metropole: Köln
JSON-Datei für Köln wurde erfolgreich unter export/data_köln.json gespeichert.
Verarbeite Metropole: München
JSON-Datei für München wurde erfolgreich unter export/data_münchen.json gespeichert.
Verarbeite Metropole: Frankfurt
JSON-Datei für Frankfurt wurde erfolgreich unter export/data_frankfurt.json gespeichert.
Verarbeite Metropole: Hamburg
JSON-Datei für Hamburg wurde erfolgreich unter export/data_hamburg.json gespeichert.


### Calculation

Hier wird eine später im Frontend stattfindende Berechnung des Scores aufgrund der User-Werte simuliert. Es dient als Anschauungsbeispiel für die Frontend-Programierung.

##### User Input

Hier wird der User-Input defininert, wobei drei Inputtypen unterschieden werden: kontinuierliche Werte (Preis), kategoriale Werte (Angebot, Schule, Pendler) und Boolean-Werte (Autobahn, Zug, Supermarkt). 

Erklärung: "None" bedeutet, dass diese Variable vom User nicht gesetzt wurde und in der Berechnung nicht berücksichtigt wird. Kontinuum-Werte können jeden erdenklichen Wert annehmen. Kategorie-Werte könenn -1, 0 und 1 sein, die für die klickbaren Label im Interface stehen (z.B. "fallend", "neutral" und "steigend"). Boolean-Werte sind entweder "true" oder None. 

In [14]:
# User-Inputs definieren
user_inputs = {
    'preis': None,  # integriert (Werte: Kontinuum)
    'angebot': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'preis_entwicklung': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'angebot_entwicklung': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'autobahn': None,  # integriert (Werte: Boolean / true, false)
    'zug': None,  # integriert (Werte: Boolean / true, false)
    'supermarkt': None,  # integriert (Werte: Boolean / true, false)
    'pendler': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'schule': 1,  # integriert (Werte: Kategorie / -1, 0, 1)
    'einwohner': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'gesundheit': None,  # integriert (Werte: Boolean / true, false)
}

# Anzahl der gültigen Inputs berechnen
valid_inputs = {key: value for key, value in user_inputs.items() if value is not None}
anzahl_der_inputs = len(valid_inputs)
print(anzahl_der_inputs)

1


##### Scaling of User Input

Bei kontinuerlichen Inputs müssen die User-Inputs komplett mit den Varianz-Werten der vorherigen Skalierung skaliert werden. Die dafür notwendigen Werte müssen dem JSON entnommen werden. Kategoriale Inputs und Boolean-Werte dürfen nicht skaliert werden. 

In [15]:
# Skalierte User-Inputs berechnen mit StandardScaler
scaled_inputs = {}
for column, user_value in valid_inputs.items():
    if column in ['autobahn', 'zug', 'supermarkt', 'gesundheit']:
        # Boolean-Werte werden in True/False umgewandelt
        scaled_inputs[column] = True if user_value == "true" else False
    elif column in columns_to_standardize and column not in ['angebot', 'preis_entwicklung', 'angebot_entwicklung', 'schule', 'pendler', 'einwohner']:
        # Index der Spalte im StandardScaler
        column_index = columns_to_standardize.index(column)
        # Skalierung des User-Inputs mit den gespeicherten Mittelwerten und Varianzen
        scaled_inputs[column] = (user_value - scaler.mean_[column_index]) / np.sqrt(scaler.var_[column_index])
    else:
        # Kategoriale Werte direkt übernehmen (unskaliert)
        scaled_inputs[column] = user_value

# Ergebnis anzeigen
print("Skalierte User-Inputs:")
for column, scaled_value in scaled_inputs.items():
    print(f"{column}: {scaled_value}")

Skalierte User-Inputs:
schule: 1


##### Score Calculation & Normalization

Bei der Berechnung des Scores wird nach den drei Input-Typen unterschieden. Im Anschluss wird der Score normiert, wobei 1 der Bestwert ist, um eine Vergleichbarkeit zu ermöglichen. 

In [17]:
# Formel anwenden
df_scaled = filtered_df_scaled.copy()
df_scaled['score'] = 1 - sum(
    (
        # Kategoriale Werte behandeln
        abs(df_scaled[column] - df_scaled[column].median()) / abs(df_scaled[column].median())
        if column in ['angebot', 'preis_entwicklung', 'angebot_entwicklung', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == 0 else
        (df_scaled[column] - df_scaled[column].min()) / -(df_scaled[column].min())  
        if column in ['angebot', 'preis_entwicklung', 'angebot_entwicklung', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == -1 else
        (df_scaled[column] - df_scaled[column].max()) / -(df_scaled[column].max())  
        if column in ['angebot', 'preis_entwicklung', 'angebot_entwicklung', 'schule', 'pendler', 'einwohner'] and scaled_inputs[column] == 1 else
        # Boolean-Werte behandeln
        (df_scaled[column].min() - df_scaled[column]) / df_scaled[column].min()  
        if column in ['autobahn', 'zug', 'supermarkt'] and scaled_inputs[column] == True else
        (df_scaled[column].min() - df_scaled[column]) / -df_scaled[column].min()  
        if column in ['gesundheit'] and scaled_inputs[column] == True else
        # Kontinuierliche Werte behandeln
        abs((df_scaled[column] - scaled_inputs[column])) / (abs(scaled_inputs[column]))
    )
    for column in scaled_inputs
) / anzahl_der_inputs

# Min-Max-Normalisierung des Scores
score_min = df_scaled['score'].min()
score_max = df_scaled['score'].max()
df_scaled['score_normalized'] = (df_scaled['score'] - score_min) / (score_max - score_min)

# Ergebnis anzeigen
df_scaled[['id', 'schule', 'score', 'score_normalized']].head(10)

,id,schule,score,score_normalized
25,2000,-1.122653,-0.603608,0.036150
26,3353,-0.448000,-0.240873,0.254172
30,1053,-0.323616,-0.173996,0.294368
35,1056,1.184959,0.637107,0.781883
36,1060,-0.105164,-0.056543,0.364964
37,1062,-1.234515,-0.663752,0.000000
51,3355,0.189084,0.101663,0.460054
95,3359,1.859906,1.000000,1.000000


In [21]:
filtered_df.head(10)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,einwohner,gesundheit,schule
25,2000,Hamburg,Hamburg,5799,4837.0,2.401554,-29.076246,4590.0,2440.0,790.0,3.244329,6.765029,61.9,34.709134
26,3353,Harburg,Hamburg,3331,895.0,0.604047,-27.176566,5950.0,5000.0,2190.0,45.054418,10.068261,59.3,37.554625
30,1053,Herzogtum Lauenburg,Hamburg,3111,646.0,0.842788,-31.130064,8880.0,6750.0,3340.0,32.132998,9.334504,62.7,38.079244
35,1056,Pinneberg,Hamburg,3558,1439.0,1.831712,-16.337209,5370.0,2600.0,1950.0,41.805565,8.344655,61.3,44.441975
36,1060,Segeberg,Hamburg,3303,993.0,3.900598,-27.571116,5920.0,3830.0,2790.0,31.295053,7.740813,56.8,39.000609
37,1062,Stormarn,Hamburg,3655,894.0,1.952580,-29.439621,5230.0,4990.0,2120.0,43.743886,5.706042,59.9,34.237333
51,3355,Lüneburg,Hamburg,3184,626.0,2.942127,-26.869159,10230.0,6220.0,2810.0,17.160771,2.142450,63.7,40.241662
95,3359,Stade,Hamburg,2659,749.0,-1.262532,-25.174825,9100.0,4320.0,2640.0,27.331942,6.068067,54.9,47.288702


In [22]:
print(filtered_df['schule'].min())

34.2373331936987


In [20]:
print(score_max)

1.0
